In [40]:
import os
import numpy as np
import pandas as pd
import torch
import os.path as osp
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle
IMG_DIR = '/mnt/chicm/data/open-images/train/imgs'
MASK_DIR = '/mnt/chicm/data/open-images/masks/train'
DATA_DIR = '/mnt/chicm/data/open-images/segmentation'

In [41]:
def get_top_classes(start_index, end_index):
    df = pd.read_csv('top_classes_level1.csv')
    c = df['class'].values[start_index:end_index]
    #print(df.head())
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi

In [42]:
classes_0_20 = set(get_top_classes(0, 20)[0])
classes_20_100 = set(get_top_classes(20, 100)[0])
classes_100_275 = set(get_top_classes(100, 275)[0])
#classes_0_20
classes_275 = set(get_top_classes(0, 275)[0])

In [43]:
len(classes_0_20)

20

In [23]:
#df = pd.read_csv(osp.join(DATA_DIR, 'sub_0902_3_top50.csv.zip'))
#df.head()

In [79]:
#df = pd.read_csv('../seg/ens_htc275_4619_od1000_0928.csv')
#df = pd.read_csv('../seg/merge_0930_7model_top120_od50_new_lb5114.csv')
#df = pd.read_csv('../seg/merge_0930_9models_lb5145.csv')
#df = pd.read_csv('../seg/lb5145/ens_0_20_5145_0265_all.csv')
df = pd.read_csv('od628_mask_300_top50.csv')


In [80]:
df.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,00000b4dcff7f799,1024,683,/m/07r04 0.9051 eNqLjzUNSDCINjUYBaNgFIyCUTBYQE...
1,00001a21632de752,1024,681,/m/03bt1vf 0.6542 eNrLSM4yDbUIsTUYBSMNpMaFWwEA...
2,0000d67245642c5f,1024,683,/m/079cl 0.2559 eNrLCcozjk4xCDA0GAWjYBSMglEwCu...
3,0001244aa8ed3099,1024,683,/m/0c9ph5 0.2090 eNqLic+wTrNOtTQYBaNgFAweEBZoC...
4,000172d1dd1adce0,1024,682,/m/04h7h 0.2846 eNpLT4gyjUo0CDAxGAWjYBSMAtqB6I...


In [143]:
#df = pd.read_csv('od_convert_0928_top1000_275.csv')

In [81]:
df.PredictionString = df.PredictionString.fillna('')
df.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,00000b4dcff7f799,1024,683,/m/07r04 0.9051 eNqLjzUNSDCINjUYBaNgFIyCUTBYQE...
1,00001a21632de752,1024,681,/m/03bt1vf 0.6542 eNrLSM4yDbUIsTUYBSMNpMaFWwEA...
2,0000d67245642c5f,1024,683,/m/079cl 0.2559 eNrLCcozjk4xCDA0GAWjYBSMglEwCu...
3,0001244aa8ed3099,1024,683,/m/0c9ph5 0.2090 eNqLic+wTrNOtTQYBaNgFAweEBZoC...
4,000172d1dd1adce0,1024,682,/m/04h7h 0.2846 eNpLT4gyjUo0CDAxGAWjYBSMAtqB6I...


In [83]:
#selected_rel = ['plays', 'inside_of', 'holds']
#selected_rel = 'on'
def get_filtered_pred_string(pred_str, selected_classes):
    if len(pred_str) < 1:
        return ''
    dets = []
    det = []
    for i, e in enumerate(pred_str.split(' ')):
        if i % 3 == 0:
            det = []
        det.append(e)
        if (i+1) % 3 == 0: # and det[0] in selected_classes:
            det[1] = str(float(det[1]) / 1000) # low conf for concat
            dets.append(' '.join(det))
                
    return ' '.join(dets)

In [72]:
#selected_rel = ['plays', 'inside_of', 'holds']
#selected_rel = 'on'
def get_top_masks(pred_str, max_num=50):
    if len(pred_str) < 1:
        return ''
    
    return ' '.join(pred_str.split()[:(max_num*3)])

In [84]:
df.PredictionString = df.PredictionString.map(lambda x: get_filtered_pred_string(x, classes_0_20))

In [73]:
df.PredictionString = df.PredictionString.map(lambda x: get_top_masks(x))

In [85]:
df.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,00000b4dcff7f799,1024,683,/m/07r04 0.0009051 eNqLjzUNSDCINjUYBaNgFIyCUTB...
1,00001a21632de752,1024,681,/m/03bt1vf 0.0006542 eNrLSM4yDbUIsTUYBSMNpMaFW...
2,0000d67245642c5f,1024,683,/m/079cl 0.00025590000000000004 eNrLCcozjk4xCD...
3,0001244aa8ed3099,1024,683,/m/0c9ph5 0.00020899999999999998 eNqLic+wTrNOt...
4,000172d1dd1adce0,1024,682,/m/04h7h 0.00028460000000000003 eNpLT4gyjUo0CD...


In [86]:
len(df.iloc[0].PredictionString.split())

150

In [19]:
145*3

435

In [87]:
#df.to_csv('../seg/merge_0930_9models_lb5145_0_20.csv', index=False)
#df.to_csv('sub_od_convert_0928_top300_275.csv', index=False)
#df.to_csv('../seg/lb5145/ens_0_20_5145_0265_top50.csv', index=False)
df.to_csv('od628_mask_300_top50_lowconf1000.csv', index=False)

In [24]:
#!kaggle competitions submit -c open-images-2019-instance-segmentation -f sub_4195_20_100.csv -m "20_100"

100%|████████████████████████████████████████| 440M/440M [00:28<00:00, 16.0MB/s]
Successfully submitted to Open Images 2019 - Instance Segmentation

In [25]:
len(classes_100_276)

175

In [41]:
!ls

check_preds.ipynb	  sub_4195_100_275.csv	top_classes.csv
check_preds-parent.ipynb  sub_4195_100_276.csv	top_classes_level1.csv
eda.ipynb		  sub_4195_20_100.csv	top_classes_parent.csv
filter_classes.ipynb	  submit.ipynb
merge_df.ipynb		  submit-parent.ipynb


In [42]:
!md5sum sub_4195*

e28ce33501f39d8492276b1c4f4d9ca7  sub_4195_100_275.csv
e28ce33501f39d8492276b1c4f4d9ca7  sub_4195_100_276.csv
f6a517f5b5888dfa28f125ebddd39ac0  sub_4195_20_100.csv


# check top 5000

In [58]:
df = pd.read_csv('sub_4195_20_100.csv')
df_test = pd.read_csv(osp.join(DATA_DIR, 'sample_empty_submission.csv'))

In [59]:
df.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/07r04 0.0049228 eNrNU0tvwyAM/kvh1Sw5RMphbdM...
1,52a76f3fe21185bf,1024,683,/m/01599 0.1188553 eNqlk9lymzAUhl9JCwjiSabT6Tg...
2,65e9da7c04c8e4cd,1024,682,NaN
3,6d10fa21d93bd6cc,1024,682,/m/01n4qj 0.0017988 eNpLiA2zMwpJNTDxN4RAAwMYCw...
4,0841dee573a875ca,1024,683,/m/015qff 0.0016251 eNpVUGt3gjAM/UtpsfgY5Ux0Pi...


In [60]:
df_test.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,-1,-1,NaN
1,52a76f3fe21185bf,-1,-1,NaN
2,65e9da7c04c8e4cd,-1,-1,NaN
3,6d10fa21d93bd6cc,-1,-1,NaN
4,0841dee573a875ca,-1,-1,NaN


In [61]:
df_5000 = pd.concat([df.iloc[:5000].copy(), df_test.iloc[5000:].copy()])
df_5000.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/07r04 0.0049228 eNrNU0tvwyAM/kvh1Sw5RMphbdM...
1,52a76f3fe21185bf,1024,683,/m/01599 0.1188553 eNqlk9lymzAUhl9JCwjiSabT6Tg...
2,65e9da7c04c8e4cd,1024,682,NaN
3,6d10fa21d93bd6cc,1024,682,/m/01n4qj 0.0017988 eNpLiA2zMwpJNTDxN4RAAwMYCw...
4,0841dee573a875ca,1024,683,/m/015qff 0.0016251 eNpVUGt3gjAM/UtpsfgY5Ux0Pi...


In [62]:
df_5000.tail()

,ImageID,ImageWidth,ImageHeight,PredictionString
99994,e40f388b3a4f80a6,-1,-1,NaN
99995,a21b4cd415d83b4d,-1,-1,NaN
99996,750b5d6e41c23bc9,-1,-1,NaN
99997,4944489aca12f92e,-1,-1,NaN
99998,632dad5b912b4b6a,-1,-1,NaN


In [63]:
df_5000.to_csv('sub_4195_20_100_5000.csv', index=False)

In [64]:
!kaggle competitions submit -c open-images-2019-instance-segmentation -f sub_4195_20_100_5000.csv -m "test 5000"

100%|██████████████████████████████████████| 24.2M/24.2M [00:13<00:00, 1.85MB/s]
Successfully submitted to Open Images 2019 - Instance Segmentation

In [129]:
df.iloc[0].PredictionString

'/m/07r04 0.0008837000000000001 eNrLCjILTDCIMjUYBaNgFIyCUTBwICs1xgwAQ5sMww== /m/04yx4 0.0007584 eNqLzYpOMkixSrcyGLogLzPfAAC0qyd6 /m/0fly7 0.0007466 eNrLys9MMgg2jUgwGECQmB9mCACpCBus /m/04yx4 0.0007093000000000001 eNpLCkq0TLTMsjYYiiArMccSAIFTJe4= /m/04yx4 0.0007056 eNqLywpOMIiyCrQ2GG4gJD7AGAAWmCsy /m/083wq 0.0006318000000000001 eNpLygw0STLNtDcYBfhBRFa4HQCKYToX /m/012n7d 0.0006284999999999999 eNoLDDZJts8wNRgFo2AUjAJ6gPTAGEsASrfX1Q== /m/04yx4 0.0006114 eNpLCQm1STVOSzQYFiAgNNgcACvdKPg= /m/083wq 0.0005924000000000001 eNoLSjDIMYxPNhgQEBUflGIAALclGgE= /m/01jfm_ 0.0005331 eNoLt8s3iEkxIBGE5WSmGAAAfYcMtA== /m/083wq 0.00040239999999999997 eNoLzUsyTjdISTGgKYgOzEgwAAC8PBX8 /m/04yx4 0.0004011 eNrLzAy1ibHMtzYYniAjPscMAPXmLFw= /m/04yx4 0.00038679999999999997 eNqLDc1MMIg1zks0GFQgOzPQGACqCB8D /m/083wq 0.0003846 eNrLSw5IMcgySkwyoAgEBaYaAAB7Rg/t /m/0fly7 0.0003725 eNrLzMm3jDXJSzCgA0gLirQCADi0Fug= /m/0fly7 0.00030819999999999996 eNqLz09MMAg0i7I3GHwgIz/MGAAr6h/Q /m/07r04 0.0003008 eNpLzkhPMgiwiLY1GDkgIdsYAJe